In [1]:
import psycopg2
import pandas as pd

In [2]:
def create_database():
    # connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres \
                            user=postgres password=manager")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS insurance")
    cur.execute("CREATE DATABASE insurance")
    
    # close connection to default database
    conn.close()
    
    # connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=insurance \
                            user=postgres password=manager")
    cur = conn.cursor()
    
    return cur, conn

In [3]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [5]:
ClaimData = pd.read_csv("data/Train_Claim.csv")

In [6]:
ClaimData.head()

,CustomerID,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,PropertyDamage,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage
0,Cust10000,2015-02-03,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City1,Location 1311,17,3,?,1,0,?,65501,13417,6071,46013
1,Cust10001,2015-02-02,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City5,Location 1311,10,3,YES,2,1,YES,61382,15560,5919,39903
2,Cust10002,2015-01-15,Single Vehicle Collision,Side Collision,Minor Damage,Other,State8,City6,Location 2081,22,1,YES,2,3,NO,66755,11630,11630,43495
3,Cust10003,2015-01-19,Single Vehicle Collision,Side Collision,Minor Damage,Other,State9,City6,Location 2081,22,1,YES,2,3,NO,66243,12003,12003,42237
4,Cust10004,2015-01-09,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State8,City6,Location 1695,10,1,NO,2,1,YES,53544,8829,7234,37481


In [7]:
ClaimData_clean = ClaimData[['CustomerID','TypeOfIncident','TypeOfCollission',
                             'IncidentState','AmountOfTotalClaim']]

In [8]:
ClaimData_clean.head()

,CustomerID,TypeOfIncident,TypeOfCollission,IncidentState,AmountOfTotalClaim
0,Cust10000,Multi-vehicle Collision,Side Collision,State7,65501
1,Cust10001,Multi-vehicle Collision,Side Collision,State7,61382
2,Cust10002,Single Vehicle Collision,Side Collision,State8,66755
3,Cust10003,Single Vehicle Collision,Side Collision,State9,66243
4,Cust10004,Single Vehicle Collision,Rear Collision,State8,53544


In [9]:
DemographicsData = pd.read_csv("data/Train_Demographics.csv")

In [10]:
DemographicsData.head()

,CustomerID,InsuredAge,InsuredZipCode,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies,CapitalGains,CapitalLoss,Country
0,Cust10000,35,454776,MALE,JD,armed-forces,movies,56700,-48500,India
1,Cust10001,36,454776,MALE,JD,tech-support,cross-fit,70600,-48500,India
2,Cust10002,33,603260,MALE,JD,armed-forces,polo,66400,-63700,India
3,Cust10003,36,474848,MALE,JD,armed-forces,polo,47900,-73400,India
4,Cust10004,29,457942,FEMALE,High School,exec-managerial,dancing,0,-41500,India


In [11]:
DemographicsData.columns

Index(['CustomerID', 'InsuredAge', 'InsuredZipCode', 'InsuredGender',
       'InsuredEducationLevel', 'InsuredOccupation', 'InsuredHobbies',
       'CapitalGains', 'CapitalLoss', 'Country'],
      dtype='object')

In [12]:
DemographicsData = DemographicsData.drop(['InsuredZipCode','CapitalGains','CapitalLoss','Country'],axis=1)

In [13]:
DemographicsData.head()

,CustomerID,InsuredAge,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies
0,Cust10000,35,MALE,JD,armed-forces,movies
1,Cust10001,36,MALE,JD,tech-support,cross-fit
2,Cust10002,33,MALE,JD,armed-forces,polo
3,Cust10003,36,MALE,JD,armed-forces,polo
4,Cust10004,29,FEMALE,High School,exec-managerial,dancing


In [14]:
PolicyData = pd.read_csv("data/Train_Policy.csv")

In [15]:
PolicyData.columns

Index(['InsurancePolicyNumber', 'CustomerLoyaltyPeriod',
       'DateOfPolicyCoverage', 'InsurancePolicyState',
       'Policy_CombinedSingleLimit', 'Policy_Deductible',
       'PolicyAnnualPremium', 'UmbrellaLimit', 'InsuredRelationship',
       'CustomerID'],
      dtype='object')

In [16]:
PolicyData = PolicyData[['CustomerLoyaltyPeriod','InsurancePolicyState',
                         'InsuredRelationship','CustomerID']]

In [17]:
PolicyData.head()

,CustomerLoyaltyPeriod,InsurancePolicyState,InsuredRelationship,CustomerID
0,328,State3,husband,Cust1001
1,256,State1,unmarried,Cust1004
2,228,State1,unmarried,Cust1005
3,256,State3,unmarried,Cust1006
4,137,State2,husband,Cust1007


In [18]:
cur, conn = create_database()

In [19]:
claim_table_create = ("""CREATE TABLE IF NOT EXISTS claimdata(
CustomerID VARCHAR PRIMARY KEY,
TypeOfIncident VARCHAR,
TypeOfCollission VARCHAR,
IncidentState VARCHAR,
AmountOfTotalClaim VARCHAR
)""")

In [20]:
cur.execute(claim_table_create)
conn.commit()

In [21]:
demographics_table_create = ("""CREATE TABLE IF NOT EXISTS demographicsdata(
CustomerID VARCHAR,
InsuredAge VARCHAR,
InsuredGender VARCHAR,
InsuredEducationLevel VARCHAR,
InsuredOccupation VARCHAR,
InsuredHobbies VARCHAR
)""")

cur.execute(demographics_table_create)
conn.commit()

In [22]:
policy_table_create = ("""CREATE TABLE IF NOT EXISTS policydata(
CustomerLoyaltyPeriod INT ,
InsurancePolicyState VARCHAR,
InsuredRelationship VARCHAR,
CustomerID VARCHAR
)""")

In [23]:
claim_table_insert = ("""INSERT INTO claimdata(
CustomerID,
TypeOfIncident,
TypeOfCollission,
IncidentState,
AmountOfTotalClaim)
VALUES (%s,%s,%s,%s,%s)
""")

In [24]:
for i, row in ClaimData_clean.iterrows():
    cur.execute(claim_table_insert, list(row))
    #     print(list(row))

In [25]:
conn.commit()

In [26]:
demographics_table_insert = ("""INSERT INTO demographicsdata(
CustomerID ,
InsuredAge ,
InsuredGender,
InsuredEducationLevel ,
InsuredOccupation ,
InsuredHobbies)
VALUES (%s,%s,%s,%s,%s,%s)
""")

In [27]:
for i, row in DemographicsData.iterrows():
    cur.execute(demographics_table_insert, list(row))

In [28]:
conn.commit()

In [29]:
policy_table_insert = ("""INSERT INTO policydata(
CustomerLoyaltyPeriod ,
InsurancePolicyState ,
InsuredRelationship ,
CustomerID)
VALUES (%s,%s,%s,%s)
""")

In [30]:
for i, row in PolicyData.iterrows():
    cur.execute(policy_table_insert, list(row))

In [31]:
conn.commit()